In [2]:
import pandas as pd
import numpy as np

In [3]:
titanic=pd.read_csv("titanic.csv")

In [4]:
titanic # 이 엄청난 데이터를 출력한다

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [5]:
titanic.isna().sum() # 결측값이 어디어디 있나 확인

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [6]:
titanic.info()
# age(나이) : float64
# deck(선박 위치, 형태) : object
# embark_town(탑승한 항구) : object 임을 확인
# embarked(탑승한 항구) : object 임을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [7]:
for col in ("age","deck","embark_town","embarked"):
    # 숫자 타입이라면 0.0(float64) 형태로 바꿔주기
    if col=="age" : titanic[col]=titanic[col].fillna(0.0)
    else :
        # str 타입이라면 "None"형태로 바꿔주기
        titanic[col]=titanic[col].fillna("None")
titanic.isna().sum() # 결측값 제거 확인 완료

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [50]:
#1 : 사망자별 나잇대 분석
death=titanic["survived"]==1
buckets=[0,5,12,19,64,titanic["age"].max()]
# 죽지 않은 사람들의 DataFrame에서 "age"열만 갖고온 후(Series객체), bins 사용
titanic[death]["age"].value_counts(bins=buckets, sort=False).to_frame() # bins는 Series에서만 된다!! 가슴에 새기자....胸に刻もう。。
# 나잇대를
# 유아(0~5세) : 어른의 품에 안겨 많이 살아남았음을 추정할 수 있다
# 어린이(6~12세) : 많이 살아남지 못했다...
# 청소년(13~19세) : 청소년들은 많이 살아남았다 
# 성인(20~64세) : 성인도 많이 살아남았다
# 노인(65세 이상) : 살아남으신 한 분, 대단하십니다..어르신

,count
age,
"(-0.001, 5.0]",83
"(5.0, 12.0]",9
"(12.0, 19.0]",39
"(19.0, 64.0]",210
"(64.0, 80.0]",1


In [54]:
#1 : 나잇대별 사람들이 몇 명 있는지 확인하고, 비율로 나눠보자
death_list=titanic[death]["age"].value_counts(bins=buckets, sort=False).to_list()
every_list=titanic["age"].value_counts(bins=buckets, sort=False).to_list()
for idx, value in enumerate(["유아기","아동기","청소년기","성인","노년기"]) :
    print(f'{value} 생존 비율 : {round((death_list[idx]/every_list[idx])*100,2)} %')
# 청소년기 -> 성인 -> 유아기 -> 아동기 -> 노년기 순으로 많이 살아남았음을 알 수 있다

유아기 생존 비율 : 37.56 %
아동기 생존 비율 : 36.0 %
청소년기 생존 비율 : 41.05 %
성인 생존 비율 : 38.96 %
노년기 생존 비율 : 9.09 %


In [31]:
#2 어떤 등급의 좌석에서 사람이 상대적으로 많이 죽었나
death=titanic["survived"]==0
titanic[death]["class"].value_counts(sort=False)
# 이런 젠장! 역시 돈이 최고다.....
# 근데 잠깐, 일등석과 삼등석이 몇개인줄 알고?

class
Third     372
First      80
Second     97
Name: count, dtype: int64

In [32]:
#2 일등석과 삼등석의 좌석개수를 보자
titanic["class"].value_counts(sort=False)
# 예상대로 삼등석이 많다. 그래도 너무 많이 죽었는데? 비율을 알아보자

class
Third     491
First     216
Second    184
Name: count, dtype: int64

In [37]:
#2
death_count=titanic[death]["class"].value_counts(sort=False).to_list()
class_count=titanic["class"].value_counts(sort=False).to_list()  # value_count() 결과값이 뒤섞임을 방지하여 sort=False를 넣어 행 순서를 일치시킴!
for idx, value in enumerate(["삼등석","일등석","이등석"]) :
    print(f'{value} 사망 비율 : {round((death_count[idx]/class_count[idx])*100,2)} %')
# 칙쇼~~역시 돈이 좋다........배를 탈때는 일등석! 삼등석은 사망 비율이 75%나 된다 ㄷㄷ 어떻게 되먹은겨!

삼등석 사망 비율 : 75.76 %
일등석 사망 비율 : 37.04 %
이등석 사망 비율 : 52.72 %


In [38]:
#3 이쯤되면 죽음과 다른 요인의 관계성을 파악하는 분석이 될 것 같아서, 똑같은 분석에 질리는 마음이 든다
# 여성, 남성별 평균 나잇대를 파악해보자
# 음? 아이도 있다! 아이는 성별 파악을 안하는 건가...뭐하는 데이터인가!
titanic["who"].unique()

array(['man', 'woman', 'child'], dtype=object)

In [46]:
print(f'남자 : {titanic[titanic["who"]=="man"]["who"].shape[0]}')
print(f'여자 : {titanic[titanic["who"]=="woman"]["who"].shape[0]}')
print(f'아이들 : {titanic[titanic["who"]=="child"]["who"].shape[0]}')
# 남자가 많이 탔다

남자 : 537
여자 : 271
아이들 : 83
